<a href="https://colab.research.google.com/github/roktavia-cmd/Grok/blob/main/grok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('dataset_twt.csv')

In [4]:
print(df.head())

   bookmarks  conversation_id                      created_at  \
0       7293     2.003841e+18  Wed Dec 24 14:52:38 +0000 2025   
1        631     2.007191e+18  Fri Jan 02 20:42:30 +0000 2026   
2          0     2.013124e+18  Mon Jan 19 05:40:07 +0000 2026   
3          0     2.014579e+18  Fri Jan 23 06:01:02 +0000 2026   
4       1078     2.007043e+18  Fri Jan 02 10:54:54 +0000 2026   

   entities/hashtags/0/indices/0  entities/hashtags/0/indices/1  \
0                            NaN                            NaN   
1                            NaN                            NaN   
2                            NaN                            NaN   
3                            NaN                            NaN   
4                            NaN                            NaN   

  entities/hashtags/0/text  entities/hashtags/1/indices/0  \
0                      NaN                            NaN   
1                      NaN                            NaN   
2                      

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Columns: 491 entries, bookmarks to views
dtypes: bool(1), float64(350), int64(11), object(129)
memory usage: 145.6+ KB
None


In [6]:
for col in df.columns :
  print(col)

bookmarks
conversation_id
created_at
entities/hashtags/0/indices/0
entities/hashtags/0/indices/1
entities/hashtags/0/text
entities/hashtags/1/indices/0
entities/hashtags/1/indices/1
entities/hashtags/1/text
entities/hashtags/2/indices/0
entities/hashtags/2/indices/1
entities/hashtags/2/text
entities/hashtags/3/indices/0
entities/hashtags/3/indices/1
entities/hashtags/3/text
entities/media/0/additional_media_info/monetizable
entities/media/0/allow_download_status/allow_download
entities/media/0/display_url
entities/media/0/expanded_url
entities/media/0/ext_media_availability/status
entities/media/0/features/all/tags/0/name
entities/media/0/features/all/tags/0/screen_name
entities/media/0/features/all/tags/0/type
entities/media/0/features/all/tags/0/user_id
entities/media/0/features/large/faces/0/h
entities/media/0/features/large/faces/0/w
entities/media/0/features/large/faces/0/x
entities/media/0/features/large/faces/0/y
entities/media/0/features/medium/faces/0/h
entities/media/0/featur

In [9]:
df_clean = df[['created_at', 'text', 'views', 'favorites', 'retweets', 'user_info/screen_name']].copy()
df_clean['created_at'] = pd.to_datetime(df_clean['created_at'], errors = 'coerce')
print(df_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   created_at             38 non-null     datetime64[ns, UTC]
 1   text                   38 non-null     object             
 2   views                  38 non-null     int64              
 3   favorites              38 non-null     int64              
 4   retweets               38 non-null     int64              
 5   user_info/screen_name  38 non-null     object             
dtypes: datetime64[ns, UTC](1), int64(3), object(2)
memory usage: 1.9+ KB
None


/tmp/ipython-input-4205279153.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean['created_at'] = pd.to_datetime(df_clean['created_at'], errors = 'coerce')


In [19]:
import re
def bersihkan_tweet(teks):
  teks =re.sub(r'@[A-Za-z0-0_]+', '', teks)
  teks =re.sub(r'#', '', teks)
  teks =re.sub(r'https?://\s+|www\.\S+', '', teks)
  teks =re.sub(r'[^a-zA-Z\s]', '', teks)
  teks =teks.lower().strip()
  return teks


In [20]:
df_clean['text_clean'] = df_clean['text'].apply(bersihkan_tweet)
print(df_clean[['text', 'text_clean']].head())

                                                text  \
0  If you don't want your posts used by Grok and ...   
1  Apparently this is how you can stop people fro...   
2  In case folks didn't know you can edit your se...   
3  Hey @Grok, could you give Dubai-RUSSELL more p...   
4  Entrad en "configuración de privacidad". Luego...   

                                          text_clean  
0  if you dont want your posts used by grok and o...  
1  apparently this is how you can stop people fro...  
2  in case folks didnt know you can edit your set...  
3  hey  could you give dubairussell more privacy ...  
4  entrad en configuracin de privacidad luego en ...  


In [21]:
from collections import Counter
semua_kata = " ".join(df_clean['text_clean']).split()
hitung_kata = Counter(semua_kata)
print(hitung_kata.most_common(10))

[('and', 64), ('the', 37), ('grok', 36), ('to', 35), ('privacy', 29), ('your', 27), ('is', 24), ('this', 23), ('you', 21), ('in', 18)]


In [22]:
from itertools import count
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stop_words.update(['rt', 'via', 'amp'])

kata_bermakna = [kata for kata in semua_kata if kata not in stop_words and len(kata) > 2]
hitung_kata_baru = Counter(kata_bermakna)
print(hitung_kata_baru.most_common(10))

[('grok', 36), ('privacy', 29), ('safety', 15), ('turn', 12), ('settings', 11), ('data', 10), ('also', 8), ('used', 7), ('scroll', 6), ('make', 6)]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
df_final = df_clean[['created_at', 'text_clean', 'views', 'favorites', 'retweets', 'user_info/screen_name' ]].copy()

df_final.to_csv('grok_ready.csv', index=False)
print ("OK!")

OK!
